In [3]:
from statsmodels.tsa.stattools import adfuller
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from statsmodels.tsa.arima.model import ARIMA
import json

In [63]:
# f = open()
data_df = pd.read_csv('./src/api/data/mosquito_Indicator.csv')

In [64]:
data_df

,date,mosquito_Indicator,rain(mm),mean_T(℃),min_T(℃),max_T(℃)
0,2016-05-01,254.4,0.0,18.8,12.2,26.0
1,2016-05-02,273.5,16.5,21.1,16.5,28.4
2,2016-05-03,304.0,27.0,12.9,8.9,17.6
3,2016-05-04,256.2,0.0,15.7,10.2,20.6
4,2016-05-05,243.8,7.5,18.9,10.2,26.9
...,...,...,...,...,...,...
1337,2019-12-27,5.5,0.0,-1.7,-4.6,2.6
1338,2019-12-28,5.5,0.0,1.1,-3.6,6.1
1339,2019-12-29,5.5,1.4,3.8,1.1,6.2
1340,2019-12-30,5.5,0.4,2.7,-5.7,6.8


In [65]:
data_df.isna().sum()

date                  0
mosquito_Indicator    0
rain(mm)              0
mean_T(℃)             0
min_T(℃)              0
max_T(℃)              0
dtype: int64

In [66]:
data_df.index = pd.DatetimeIndex(pd.to_datetime(data_df["date"])) #.sort_index(ascending=True, inplace=False)


In [67]:
data_df

,date,mosquito_Indicator,rain(mm),mean_T(℃),min_T(℃),max_T(℃)
date,,,,,,
2016-05-01,2016-05-01,254.4,0.0,18.8,12.2,26.0
2016-05-02,2016-05-02,273.5,16.5,21.1,16.5,28.4
2016-05-03,2016-05-03,304.0,27.0,12.9,8.9,17.6
2016-05-04,2016-05-04,256.2,0.0,15.7,10.2,20.6
2016-05-05,2016-05-05,243.8,7.5,18.9,10.2,26.9
...,...,...,...,...,...,...
2019-12-27,2019-12-27,5.5,0.0,-1.7,-4.6,2.6
2019-12-28,2019-12-28,5.5,0.0,1.1,-3.6,6.1
2019-12-29,2019-12-29,5.5,1.4,3.8,1.1,6.2


In [68]:
my_range = pd.date_range(start=data_df['date'].min(), end=data_df['date'].max(), freq='D')


In [69]:
len(my_range[~my_range.isin(data_df.index)])


45

In [70]:
my_range.drop_duplicates()

DatetimeIndex(['2016-05-01', '2016-05-02', '2016-05-03', '2016-05-04',
               '2016-05-05', '2016-05-06', '2016-05-07', '2016-05-08',
               '2016-05-09', '2016-05-10',
               ...
               '2019-12-22', '2019-12-23', '2019-12-24', '2019-12-25',
               '2019-12-26', '2019-12-27', '2019-12-28', '2019-12-29',
               '2019-12-30', '2019-12-31'],
              dtype='datetime64[ns]', length=1340, freq='D')

In [71]:
print(data_df.shape)
new_df = pd.DataFrame()
new_df.index = my_range
new_df["date"] = my_range
data_df = pd.merge(data_df, new_df, how="outer", right_index=True, left_index=True, validate="many_to_many")

(1342, 6)


In [73]:
data_df.index.drop_duplicates()
print(data_df)


                date_x  mosquito_Indicator  rain(mm)  mean_T(℃)  min_T(℃)  \
2016-05-01  2016-05-01               254.4       0.0       18.8      12.2   
2016-05-02  2016-05-02               273.5      16.5       21.1      16.5   
2016-05-03  2016-05-03               304.0      27.0       12.9       8.9   
2016-05-04  2016-05-04               256.2       0.0       15.7      10.2   
2016-05-05  2016-05-05               243.8       7.5       18.9      10.2   
...                ...                 ...       ...        ...       ...   
2019-12-27  2019-12-27                 5.5       0.0       -1.7      -4.6   
2019-12-28  2019-12-28                 5.5       0.0        1.1      -3.6   
2019-12-29  2019-12-29                 5.5       1.4        3.8       1.1   
2019-12-30  2019-12-30                 5.5       0.4        2.7      -5.7   
2019-12-31  2019-12-31                 5.5       0.0       -7.9     -10.9   

            max_T(℃)     date_y  
2016-05-01      26.0 2016-05-01  
2016-05

In [74]:
data_df = data_df.groupby(data_df.index).last()

In [51]:
data_df = data_df.drop_duplicates(keep='last')

In [76]:
print(f"Duplicates: {data_df.index.duplicated()}")


Duplicates: [False False False ... False False False]


In [78]:
data_df = data_df.drop(columns=['date_x', 'date_y'])


KeyError: "['date_x', 'date_y'] not found in axis"

In [79]:
data_df["date"] = data_df.index

In [84]:
data_df.isna().sum()

mosquito_Indicator    0
rain(mm)              0
mean_T(℃)             0
min_T(℃)              0
max_T(℃)              0
date                  0
dtype: int64

In [82]:
data_df = data_df.fillna(method='ffill')

/var/folders/xk/ct1qcm4j5ljdj441cx4jphvjp9l86x/T/ipykernel_81609/516991410.py:1: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  data_df = data_df.fillna(method='ffill')


In [83]:
data_df.to_json('./src/api/data/seoulMosquitoes_25-05-24.json', orient="records")